In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install python-terrier

     |████████████████████████████████| 97 kB 3.6 MB/s 
     |████████████████████████████████| 1.1 MB 38.7 MB/s 
     |████████████████████████████████| 69 kB 7.3 MB/s 
     |████████████████████████████████| 297 kB 61.3 MB/s 
     |████████████████████████████████| 41 kB 206 kB/s 
     |████████████████████████████████| 1.2 MB 33.0 MB/s 
     |████████████████████████████████| 6.4 MB 23.9 MB/s 
     |████████████████████████████████| 596 kB 43.5 MB/s 
     |████████████████████████████████| 72 kB 1.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 126 kB 63.2 MB/s 
     |████████████████████████████████| 294 kB 53.0 MB/s 
     |████████████████████████████████| 291 kB 71.0 MB/s 
     |████████████████████████████████| 59 kB 5.8 MB/s 
     |████████████████████████████████| 45 kB 3.0 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.8.0-py3-none

In [ ]:
import pyterrier as pt
pt.init(mem=20000, boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

[INFO] trec-robust04 is deprecated. Consider using disks45/nocr/trec-robust-2004 instead, which provides better parsing of the corpus.
[INFO] trec-robust04/fold1 is deprecated. Consider using disks45/nocr/trec-robust-2004/fold1 instead, which provides better parsing of the corpus.
[INFO] trec-robust04/fold2 is deprecated. Consider using disks45/nocr/trec-robust-2004/fold2 instead, which provides better parsing of the corpus.
[INFO] trec-robust04/fold3 is deprecated. Consider using disks45/nocr/trec-robust-2004/fold3 instead, which provides better parsing of the corpus.
[INFO] trec-robust04/fold4 is deprecated. Consider using disks45/nocr/trec-robust-2004/fold4 instead, which provides better parsing of the corpus.
[INFO] trec-robust04/fold5 is deprecated. Consider using disks45/nocr/trec-robust-2004/fold5 instead, which provides better parsing of the corpus.


In [ ]:
def msmarco_generate():
    dataset = pt.get_dataset("trec-deep-learning-passages")
    with pt.io.autoopen(dataset.get_corpus()[0], 'rt') as corpusfile:
        for l in corpusfile:
            docno, passage = l.split("\t")
            yield {'docno' : docno, 'text' : passage}

iter_indexer = pt.IterDictIndexer("./passage_index", overwrite = True)
indexref = iter_indexer.index(msmarco_generate(), meta={'docno' : 20, 'text': 4096})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: specifying meta and meta_lengths in IterDictIndexer.index() is deprecated, use constructor instead
  if __name__ == '__main__':


collection.tar.gz:   0%|          | 0.00/987M [00:00<?, ?iB/s]

07:50:42.070 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (500080) - further warnings are suppressed
08:12:50.802 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 5 empty documents


In [ ]:
index = pt.IndexFactory.of(indexref)

In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('/content/albert_train.csv')
test = pd.read_csv('/content/albert_test.csv')

In [ ]:
frames = [train, test]
df = pd.concat(frames, ignore_index = True)

In [ ]:
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [ ]:
df = pt.model.add_ranks(df.astype({'qid' : 'str', 'docno' : 'str', 'score' : 'str'}))

In [ ]:
dataset = pt.datasets.get_dataset("trec-deep-learning-passages")

In [ ]:
train_topics = dataset.get_topics("train")
train_qrels = dataset.get_qrels("train")

queries.tar.gz:   0%|          | 0.00/18.0M [00:00<?, ?iB/s]

qrels.train.tsv:   0%|          | 0.00/10.1M [00:00<?, ?iB/s]

In [ ]:
train_topics = train_topics.sort_values(by='qid', ascending=True)[3:]
train_qrels = train_qrels.sort_values(by='qid', ascending=True)[3:]

In [ ]:
# A simple join operation on the qids, as we have much more queries than qrels, we want to only use those
# queries with a qrel for optimal training performances. 
temp = pd.merge(train_topics, train_qrels, left_on='qid', right_on='qid')

In [ ]:
# Make the new train_topics dataframe with only those queries that have a qrel
train_topics_100 = pd.concat([temp['qid'], temp['query']], axis=1, keys=['qid', 'query'])

In [ ]:
train_qrel_100 = pd.concat([temp['qid'], temp['docno'], temp['label']], axis=1, keys=['qid', 'docno', 'label'])

In [ ]:
import numpy as np

#This method is used to introduce a new feature in the L2R model
def features(row):
  docid = row["docid"]
  qid = row["qid"]
  num1 = 0
  num2 = 0
  f1 = df.loc[(df["qid"] == str(qid)) & (df["docno"] == str(docid))]

  if len(f1) > 0:
    num1 = f1.iloc[0]['score']

  return np.array([num1])

In [ ]:
import numpy as np

#This method is used to give a new score to a document.
#In particular, it this notebook it is used to sum the bm25 score with the distilBERT score,
#which is multiplied by a certain factor to make it more compatible with the range of bm25 scores
def scores(row):
  docid = row["docid"]
  qid = row["qid"]
  num1 = 0
  f1 = df.loc[(df["qid"] == str(qid)) & (df["docno"] == str(docid))]

  if len(f1) > 0:
    num1 = f1.iloc[0]['score']

  new_score = row['score'] + float(num1)*10

  return (new_score)

In [ ]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25", num_results=100)
bm25_1000 = pt.BatchRetrieve(index, wmodel="BM25", num_results=1000)
tf = pt.BatchRetrieve(index, wmodel="Tf")
pl2 = pt.BatchRetrieve(index, wmodel="PL2")
pipeline = (bm25 % 100) >> (pt.apply.doc_features(features) ** tf ** pl2)
L2R = (bm25 % 100) >> (tf ** pl2)
summation = bm25_1000 >> pt.apply.doc_score(scores)
albert = pt.Transformer.from_df(df)

In [ ]:
import time

start = time.time()

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=300)
pipe_albert = pipeline >> pt.ltr.apply_learned_model(rf)
pipe_albert.fit(train_topics_100[:100], train_qrel_100[:100])

end = time.time()

print(end - start)

10:15:38.026 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
681.4620888233185


In [ ]:
import time

start = time.time()

from sklearn.ensemble import RandomForestRegressor
rf_n = RandomForestRegressor(n_estimators=300)
pipe_no_albert = L2R >> pt.ltr.apply_learned_model(rf_n)
pipe_no_albert.fit(train_topics_100.head(100), train_qrel_100[:100])

end = time.time()

print(end - start)

19.22334885597229


In [ ]:
!mkdir Experiment

In [ ]:
pt.Experiment([bm25, pipe_no_albert, pipe_albert, summation, albert], dataset.get_topics("test-2019"), dataset.get_qrels("test-2019"), filter_by_qrels = True, eval_metrics=["map", "ndcg", "recip_rank"], save_dir = "/content/Experiment", names = ["bm25", "L2R", "L2R+", "sum", "albert"], baseline = 1)

msmarco-test2019-queries.tsv.gz:   0%|          | 0.00/4.18k [00:00<?, ?iB/s]

2019qrels-docs.txt:   0%|          | 0.00/183k [00:00<?, ?iB/s]

,name,map,recip_rank,ndcg,map +,map -,map p-value,recip_rank +,recip_rank -,recip_rank p-value,ndcg +,ndcg -,ndcg p-value
0,bm25,0.290681,0.794961,0.445331,34.0,7.0,5.003610e-06,13.0,5.0,0.050388,31.0,11.0,3.311705e-05
1,L2R,0.216835,0.680394,0.392542,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,L2R+,0.265180,0.843199,0.450198,33.0,7.0,6.704474e-05,14.0,4.0,0.015216,36.0,6.0,5.620159e-05
3,sum,0.429516,0.842724,0.631068,43.0,0.0,6.023537e-14,16.0,1.0,0.002733,43.0,0.0,2.331014e-16
4,albert,0.329682,0.906226,0.600923,31.0,12.0,2.353807e-03,17.0,2.0,0.000188,38.0,5.0,1.778566e-07


In [ ]:
results = pt.Experiment([bm25, pipe_no_albert, pipe_albert, summation, albert], dataset.get_topics("test-2019"), dataset.get_qrels("test-2019"), filter_by_qrels = True, eval_metrics=["map", "ndcg", "recip_rank"], save_dir = "/content/Experiment", names = ["bm25", "L2R", "L2R+", "sum", "albert"], perquery = True)

In [ ]:
DOCS = pd.read_csv("/content/drive/MyDrive/collection.tar.gz", sep='\t', header=None, names=["docid","passage"])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
results.astype({'value' : 'float'}).loc[(results['measure'] == 'ndcg') & (results['name'] == 'L2R+')].sort_values(by=['value'], ascending = False)

,name,qid,measure,value
377,L2R+,182539,ndcg,0.840299
269,L2R+,130510,ndcg,0.827071
296,L2R+,1121402,ndcg,0.790569
386,L2R+,146187,ndcg,0.725326
362,L2R+,855410,ndcg,0.723898
341,L2R+,1124210,ndcg,0.714674
365,L2R+,1114646,ndcg,0.704363
260,L2R+,156493,ndcg,0.685849
275,L2R+,573724,ndcg,0.675662
350,L2R+,87181,ndcg,0.668392


In [ ]:
pipe_albert.search("definition declaratory judgment", qid = '130510')

,qid,docid,docno,score,query,features,rank
14,130510,8612910,8612910,0.466667,definition declaratory judgment,"[0.8523081541061401, 2.0, 16.26917520354018]",0
48,130510,2961568,2961568,0.300000,definition declaratory judgment,"[0.5808991193771362, 5.0, 12.448116653157387]",1
13,130510,8612903,8612903,0.286667,definition declaratory judgment,"[0.7766799330711365, 3.0, 16.24377413170329]",2
2,130510,799647,799647,0.273333,definition declaratory judgment,"[0.7986081838607788, 5.0, 19.653569861834313]",3
0,130510,1494936,1494936,0.233333,definition declaratory judgment,"[0.7631735801696777, 6.0, 20.265028455560408]",4
...,...,...,...,...,...,...,...
95,130510,7207412,7207412,0.000000,definition declaratory judgment,"[0.0, 2.0, 10.805333954186697]",95
96,130510,7001677,7001677,0.000000,definition declaratory judgment,"[0.0, 4.0, 10.69242157339265]",96
97,130510,1063368,1063368,0.000000,definition declaratory judgment,"[0.593582808971405, 3.0, 10.47075157242548]",97
98,130510,4528808,4528808,0.000000,definition declaratory judgment,"[0.0, 2.0, 10.350643362146549]",98


In [ ]:
pipe_albert.search("why did the us volunterilay enter ww1", qid = '1063750')

,qid,docid,docno,score,query,features,rank
2,1063750,4337527,4337527,0.12,why did the us volunterilay enter ww1,"[0.6826297640800476, 3.0, 14.811733488157689]",0
0,1063750,4788295,4788295,0.00,why did the us volunterilay enter ww1,"[0.6367112398147583, 7.0, 14.982743309113086]",1
1,1063750,2997653,2997653,0.00,why did the us volunterilay enter ww1,"[0.0, 6.0, 14.65359324445613]",2
3,1063750,6093907,6093907,0.00,why did the us volunterilay enter ww1,"[0.5439461469650269, 4.0, 13.565743245806374]",3
4,1063750,2981566,2981566,0.00,why did the us volunterilay enter ww1,"[0.0, 3.0, 13.084359683460981]",4
...,...,...,...,...,...,...,...
95,1063750,3057055,3057055,0.00,why did the us volunterilay enter ww1,"[0.0, 2.0, 8.631317808871652]",95
96,1063750,3934498,3934498,0.00,why did the us volunterilay enter ww1,"[0.0, 2.0, 8.631317808871652]",96
97,1063750,5236487,5236487,0.00,why did the us volunterilay enter ww1,"[0.0, 2.0, 8.631317808871652]",97
98,1063750,5236367,5236367,0.00,why did the us volunterilay enter ww1,"[0.0, 1.0, 8.48962284810477]",98


In [ ]:
results.astype({'value' : 'float'}).loc[(results['measure'] == 'ndcg') & (results['name'] == 'sum')].sort_values(by=['value'], ascending = False)

,name,qid,measure,value
491,sum,855410,ndcg,1.000000
389,sum,156493,ndcg,0.935899
407,sum,168216,ndcg,0.932359
470,sum,1124210,ndcg,0.894750
440,sum,359349,ndcg,0.870194
398,sum,130510,ndcg,0.869912
425,sum,1121402,ndcg,0.867348
506,sum,182539,ndcg,0.860094
515,sum,146187,ndcg,0.848857
452,sum,104861,ndcg,0.839602


In [ ]:
summation.search("definition declaratory judgment", qid = 130510)

,qid,docid,docno,score,query,rank
0,130510,1494936,1494936,48.797975,definition declaratory judgment,0
2,130510,799647,799647,47.938504,definition declaratory judgment,1
3,130510,7501563,7501563,45.882586,definition declaratory judgment,2
6,130510,8612906,8612906,45.800988,definition declaratory judgment,3
1,130510,7125239,7125239,45.772413,definition declaratory judgment,4
...,...,...,...,...,...,...
995,130510,7738711,7738711,16.203673,definition declaratory judgment,995
996,130510,691377,691377,16.199872,definition declaratory judgment,996
997,130510,837069,837069,16.199872,definition declaratory judgment,997
998,130510,3913776,3913776,16.199872,definition declaratory judgment,998


In [ ]:
summation.search("why did the us volunterilay enter ww1", qid = '1063750')

,qid,docid,docno,score,query,rank
0,1063750,4788295,4788295,36.707135,why did the us volunterilay enter ww1,0
2,1063750,4337527,4337527,36.390789,why did the us volunterilay enter ww1,1
3,1063750,6093907,6093907,33.115696,why did the us volunterilay enter ww1,2
8,1063750,4337526,4337526,31.952549,why did the us volunterilay enter ww1,3
7,1063750,3633196,3633196,31.282837,why did the us volunterilay enter ww1,4
...,...,...,...,...,...,...
995,1063750,2690513,2690513,10.536260,why did the us volunterilay enter ww1,995
996,1063750,2934779,2934779,10.536260,why did the us volunterilay enter ww1,996
997,1063750,2975519,2975519,10.536260,why did the us volunterilay enter ww1,997
998,1063750,3109770,3109770,10.536260,why did the us volunterilay enter ww1,998


In [ ]:
results.astype({'value' : 'float'}).loc[(results['measure'] == 'ndcg') & (results['name'] == 'albert')].sort_values(by=['value'], ascending = False)

,name,qid,measure,value
536,albert,168216,ndcg,0.915057
551,albert,264014,ndcg,0.869842
644,albert,146187,ndcg,0.868213
539,albert,1133167,ndcg,0.862885
584,albert,833860,ndcg,0.824374
632,albert,405717,ndcg,0.823210
533,albert,573724,ndcg,0.822455
518,albert,156493,ndcg,0.799853
527,albert,130510,ndcg,0.784239
554,albert,1121402,ndcg,0.775898


In [ ]:
albert.search("definition declaratory judgment", qid = '130510')

,qid,query,docno,score,rank
0,130510,definition declaratory judgment,8612910,0.8523081541061401,0
1,130510,definition declaratory judgment,8612909,0.8017106056213379,1
2,130510,definition declaratory judgment,799647,0.7986081838607788,2
3,130510,definition declaratory judgment,8612903,0.7766799330711365,3
4,130510,definition declaratory judgment,1494936,0.7631735801696777,4
...,...,...,...,...,...
995,130510,definition declaratory judgment,5799939,0.4820962250232696,995
996,130510,definition declaratory judgment,3944906,0.4819648265838623,996
997,130510,definition declaratory judgment,1288885,0.4819608330726623,997
998,130510,definition declaratory judgment,3455182,0.4818490445613861,998


In [ ]:
albert.search("why did the us volunterilay enter ww1", qid = '1063750')

,qid,query,docno,score,rank
0,1063750,why did the us volunterilay enter ww1,6591646,0.7041118144989014,0
1,1063750,why did the us volunterilay enter ww1,7952971,0.6859273910522461,1
2,1063750,why did the us volunterilay enter ww1,7571423,0.6837289333343506,2
3,1063750,why did the us volunterilay enter ww1,4337527,0.6826297640800476,3
4,1063750,why did the us volunterilay enter ww1,5956099,0.6815410852432251,4
...,...,...,...,...,...
995,1063750,why did the us volunterilay enter ww1,7813258,0.5148706436157227,995
996,1063750,why did the us volunterilay enter ww1,279907,0.5148621201515198,996
997,1063750,why did the us volunterilay enter ww1,567943,0.5148342847824097,997
998,1063750,why did the us volunterilay enter ww1,5034251,0.5148311853408813,998
